In [4]:
import numpy as np
import pandas as pd
import pyhere

In [11]:
dir_raw_data = pyhere.here().resolve().joinpath("data", "raw")


In [14]:
csv_power_plants = pd.read_csv(dir_raw_data/"global_power_plant_database.csv")

/tmp/ipykernel_1547/1573865454.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_power_plants = pd.read_csv(dir_raw_data/"global_power_plant_database.csv")


In [37]:
sample_lat_lon = csv_power_plants[["latitude", "longitude"]].head(10)


In [38]:
list_of_tuples = list(sample_lat_lon.to_records(index=False))
list_of_tuples


[(32.322, 65.119),
 (31.67, 65.795),
 (31.623, 65.792),
 (34.556, 69.4787),
 (34.641, 69.717),
 (34.4847, 70.3633),
 (34.5638, 69.1134),
 (35.9416, 68.71),
 (34.5865, 69.7757),
 (39.9116, 20.1047)]

# MULTIPOINT

In [56]:
'''
*Version: 2.0 Published: 2021/03/09* Source: [NASA POWER](https://power.larc.nasa.gov/)
POWER API Multi-Point Download
This is an overview of the process to request data from multiple data points from the POWER API.
'''

import os, sys, time, json, urllib3, requests, multiprocessing, csv

urllib3.disable_warnings()

def download_function(collection):
    ''' '''

    request, filepath = collection
    response = requests.get(url=request, verify=False, timeout=30.00).json()

    with open(filepath, 'w') as file_object:
        json.dump(response, file_object)

def download_function_csv(collection):
    ''' '''

    request, filepath = collection
    response = requests.get(url=request, verify=False, timeout=30.00).content.decode('utf-8')

  
    
    
    with open(filepath, 'w') as file_object:
        file_object.write(response)
        file_object.close()

class Process():

    def __init__(self):

        self.processes = 5 # Please do not go more than five concurrent requests.

        self.request_template = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=CLRSKY_SFC_SW_DNI,WS10M,TQV&community=RE&longitude={longitude}&latitude={latitude}&start=20150101&end=20150305&format=CSV"
        self.filename_template = "File_Lat_{latitude}_Lon_{longitude}.csv"

        self.messages = []
        self.times = {}

    def execute(self):

        Start_Time = time.time()

        locations = list_of_tuples

        requests = []
        for latitude, longitude in locations:
            request = self.request_template.format(latitude=latitude, longitude=longitude)
            filename = self.filename_template.format(latitude=latitude, longitude=longitude)
            requests.append((request, filename))

        requests_total = len(requests)

        pool = multiprocessing.Pool(self.processes)
        x = pool.imap_unordered(download_function_csv, requests)

        for i, df in enumerate(x, 1):
            sys.stderr.write('\rExporting {0:%}'.format(i/requests_total))

        self.times["Total Script"] = round((time.time() - Start_Time), 2)

        print ("\n")
        print ("Total Script Time:", self.times["Total Script"])

if __name__ == '__main__':
    Process().execute()

Exporting 90.000000%



Total Script Time: 3.57


Exporting 100.000000%

# SINGLE POINT

In [54]:
'''
*Version: 2.0 Published: 2021/03/09* Source: [NASA POWER](https://power.larc.nasa.gov/)
POWER API Multi-Point Download
This is an overview of the process to request data from multiple data points from the POWER API.
'''

import os, json, requests

locations = [(32.929, -95.770), (5, 10)]

output = r""
base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=CLRSKY_SFC_SW_DNI,WS10M,TQV&community=RE&longitude={longitude}&latitude={latitude}&start=20150101&end=20150305&format=CSV"

for latitude, longitude in locations:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    response = requests.get(url=api_request_url, verify=True, timeout=30.00)
    
    print(response.content.decode('utf-8'))
    # content = json.loads(response.content.decode('utf-8'))
    # print(json.dumps(content,indent=2))
    # filename = response.headers['content-disposition'].split('filename=')[1]

    # filepath = os.path.join(output, filename)
    # with open(filepath, 'w') as file_object:
    #     json.dump(content, file_object)

-BEGIN HEADER-
NASA/POWER CERES/MERRA2 Native Resolution Daily Data 
Dates (month/day/year): 01/01/2015 through 03/05/2015 
Location: Latitude  32.929   Longitude -95.77 
Elevation from MERRA-2: Average for 0.5 x 0.625 degree lat/lon region = 145.83 meters
The value for missing source data that cannot be computed or is outside of the sources availability range: -999 
Parameter(s): 
CLRSKY_SFC_SW_DNI     CERES SYN1deg Clear Sky Surface Shortwave Downward Direct Normal Irradiance (kW-hr/m^2/day) 
WS10M                 MERRA-2 Wind Speed at 10 Meters (m/s) 
TQV                   MERRA-2 Total Column Precipitable Water (kg m-2) 
-END HEADER-
YEAR,MO,DY,CLRSKY_SFC_SW_DNI,WS10M,TQV
2015,1,1,4.7,2.33,28.97
2015,1,2,3.95,3.34,31.87
2015,1,3,6.6,3.23,17.66
2015,1,4,8.86,6.22,6.23
2015,1,5,8.76,3.35,5.2
2015,1,6,8.13,3.15,7.73
2015,1,7,7.57,7.17,11.96
2015,1,8,7.83,3.59,10.64
2015,1,9,7.06,4.19,13.12
2015,1,10,4.8,3.23,18.21
2015,1,11,6.18,2.62,21.98
2015,1,12,6.3,4.92,14.32
2015,1,13,6.41,3.69,